## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [1]:
import csv
from datetime import datetime

class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self._codigo = codigo
        self._nombre = nombre
        self._precio = precio
        self._tipo = tipo
        self._cantidad = cantidad

    @property
    def codigo(self):
        return self._codigo

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, valor):
        if 1 <= len(valor) <= 100:
            self._nombre = valor

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, valor):
        if 10 <= valor <= 10000:
            self._precio = valor

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, valor):
        if 0 <= valor <= 1000:
            self._cantidad = valor

    @property
    def tipo(self):
        return self._tipo

    @tipo.setter
    def tipo(self, valor):
        if 0 < len(valor) <= 20:
            self._tipo = valor

    def valorTotal(self):
        return self.precio * self.cantidad
    
    def reducir_cantidad(self, cantidad):
        if cantidad <= self._cantidad:
            self._cantidad -= cantidad
            return True
        else:
            print(f"no hay suficiente cantidad de {self.nombre} solo quedan estas {self._cantidad} unidades.")
            return False

In [ ]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [182]:
class Oferta:
    def __init__(self, tipo, valor, codigo_producto=None):
        self.tipo = tipo
        self.valor = valor
        self.codigo_producto = codigo_producto

    def calcular_desc(self, prod, cant):
        if self.tipo == "porcentaje":
            return prod.precio * cant * (self.valor / 100)
        elif self.tipo == '2x1' and cant >= 2:
            return prod.precio * (cant // 2)
        return 0

    def esAplicable(self):
        return True

    def aplicar(self, prod, cant):
        return self.calcular_desc(prod, cant)

    def descripcion(self):
        desc = ""
        if self.codigo_producto:
            desc += f"(Codigo: {self.codigo_producto}) "
        if self.tipo == "porcentaje":
            desc += f"descuento del {self.valor}%"
        elif self.tipo == '2x1':
            desc += "2x1 en la compra de lo productos"
        return desc

In [ ]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [184]:
class Catalogo:
    def __init__(self, archivo):
        self.lista_producto = []
        self.ofertas = []
        self.cargar_archivo(archivo)

    def cargar_archivo(self, archivo):
        with open(archivo, 'r') as f:
            reader = csv.DictReader(f)
            for fila in reader:
                prod = Producto(fila['codigo'], fila['nombre'], float(fila['precio']),
                                fila['tipo'], int(fila['cantidad']))
                self.lista_producto.append(prod)

    def agregar_producto(self, codigo, nombre, precio, tipo, cantidad):
        if self.buscar_producto(codigo) is not None:
            print("ya tenemos un producto con este codigo")
            return
        prod = Producto(codigo, nombre, precio, tipo, cantidad)
        self.lista_producto.append(prod)

    def buscar_producto(self, codigo):
        for p in self.lista_producto:
            if p.codigo == codigo:
                return p
        return None

    def cantidadProductos(self):
        return len(self.lista_producto)

    def cantidadUnidades(self):
        return sum(p.cantidad for p in self.lista_producto)

    def valorTotal(self):
        return sum(p.valorTotal() for p in self.lista_producto)

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscarOferta(self, producto):
        for oferta in self.ofertas:
            if (oferta.codigo_producto is None or oferta.codigo_producto == producto.codigo) and \
            (oferta.tipo == '2x1' or oferta.tipo == 'porcentaje'):
                return oferta
        return None

    def calcularDescuento(self, producto, cantidad):
        oferta = self.buscarOferta(producto, cantidad)
        if oferta:
            return oferta.aplicar(producto, cantidad)
        return 0

    def guardar(self, archivo):
        with open(archivo, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            for p in self.lista_producto:
                writer.writerow([p.codigo, p.nombre, p.precio, p.tipo, p.cantidad])
                
    def actualizar_catalogo(self, archivo):
        self.guardar(archivo)
        
    def obtener_stock(self):
        stock = {}
        for producto in self.lista_producto:
            stock[producto.codigo] = producto.cantidad
        return stock

    def generar_informe(self):
        cant_prod = self.cantidadProductos()
        total_unidades = self.cantidadUnidades()
        promedio_precio = sum(p.precio for p in self.lista_producto) / cant_prod if cant_prod > 0 else 0
        total_valor = self.valorTotal()
        tipos_prod = {}
        stock_actual = self.obtener_stock()
        
        for p in self.lista_producto:
            if p.tipo not in tipos_prod:
                tipos_prod[p.tipo] = {'unidades': 0, 'precio_total': 0}
            tipos_prod[p.tipo]['unidades'] += stock_actual[p.codigo]
            tipos_prod[p.tipo]['precio_total'] += p.valorTotal()

        print("INFORME CATALOGO")
        print(f"cantidad de productos   {cant_prod}")
        print(f"cantidad de unidades    {total_unidades}")
        print(f"precio promedio       $ {promedio_precio:.2f}")
        print(f"valor total           $ {total_valor:.2f}")
        print("tipos de productos")
        for tipo, data in tipos_prod.items():
            if data['unidades'] > 0:
                print(f"  - {tipo}              :  {data['unidades']}u x $ {data['precio_total'] / data['unidades']:.2f}")

        print("ofertas:")
        for oferta in self.ofertas:
            print(f"  - {oferta.descripcion()}")

In [ ]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [186]:
class Cliente:
    def __init__(self, nombre, apellido, cuit):
        self.nombre = nombre
        self.apellido = apellido
        if (len(cuit) == 11) or (len(cuit) == 13 and cuit[2] == '-' and cuit[11] == '-'):
            self.cuit = cuit
        else:
            print("PONE UN CUIT QUE SEA CORRECTO")
            self.cuit = None


In [ ]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

In [187]:
class Factura:
    num_factura = 1

    def __init__(self, cliente):
        self.numero = Factura.num_factura
        Factura.num_factura += 1 
        self.fecha = datetime.now().strftime("%d/%m/%Y")
        self.cliente = cliente
        self.items = []
        self.subtotal = 0
        self.descuentos = 0

    def agregar_item(self, prod, cant, oferta):
        if not prod.reducir_cantidad(cant):
            print(f"ya no se puede agregar porque no hay {cant} de {prod.nombre} stock disponible")
            return
        
        sub = prod.precio * cant
        desc = oferta.calcular_desc(prod, cant) if oferta else 0
            
        self.items.append((prod, cant, sub, desc))
        self.subtotal += sub
        self.descuentos += desc
        
        catalogo.actualizar_catalogo("catalogo.csv")

    def mostrar_factura(self):
        print(f"factura: {self.numero}")
        print(f"fecha: {self.fecha}")
        print(f"cliente: {self.cliente.nombre} {self.cliente.apellido} ({self.cliente.cuit})\n")
        for prod, cant, sub, desc in self.items:
            print(f"- {cant}u {prod.nombre}            x ${prod.precio:.2f} = ${sub:.2f}")
            if desc > 0:
                print(f"      {desc:.2f} (descuento aplicado)")
        print(f"\n                                             Subtotal:   ${self.subtotal:.2f}")
        print(f"                                             Descuentos: ${self.descuentos:.2f}")
        print(f"                                             -----------------------")
        print(f"                                             Total:      ${self.subtotal - self.descuentos:.2f}")
        
if __name__ == "__main__":
    catalogo = Catalogo("catalogo.csv")
    catalogo.generar_informe()
    
    catalogo.registrarOferta(Oferta('porcentaje', 10, '0001')) # EN ESTE CASO PUSE QUE EL PRODUCTO 0001 OSEA COCACOLA TENGA UN DESCUENTO DEL 10%
    catalogo.registrarOferta(Oferta('2x1', 0, '0002')) #Y COMO DECIA EN LA CONSIGNA PUSE QUE ALGUN PRODUCTO EN ESTE CASO 0002(PEPSI) TENGA UNA OFERTA DE 2X1

    producto1 = catalogo.buscar_producto('0001') # AQUI PUEDE PONER UN CODIGO DE ALGUN PRODUCTO
    producto2 = catalogo.buscar_producto('0002') #AQUI TAMBIEN PUEDE PONER UN CODIGO DE ALGUN PRODUCTO

    cliente = Cliente("Gaston", "Villafañe", "20-32541435-2")

    fact = Factura(cliente)
    fact.agregar_item(producto1, 2, catalogo.buscarOferta(producto1)) # EN EL NUMERO PONGA LA CANTIDAD DEL PRODUCTO1
    fact.agregar_item(producto2, 3, catalogo.buscarOferta(producto2)) # EN EL NUMERO PONGA LA CANTIDAD DEL producto2
    
    fact.mostrar_factura()
    catalogo.actualizar_catalogo("catalogo.csv")


In [ ]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

In [ ]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")